## Testing Geoapify Places API

In [1]:
import sys
import pandas as pd
import numpy as np
# change working directory to the root of the project
sys.path.append('../')
from src.geoapify_api import geoapify_pipeline
from src.methods.data_ingestion import convert_easting_northing_to_long_lat
import importlib

#### Postcode input to be converted to longitude and latitude.

Postcode data was downloaded from: https://osdatahub.os.uk/downloads/open/CodePointOpen

In [2]:
postcode_path = '../data/postcode_data/Data/'
# create function for getting postcode data
def get_postcode_data(input_postcode: str, postcode_path) -> pd.DataFrame:
    """Get postcode data from postcode csv files"""
    # match first two letters of postcode to file
    postcode_file = postcode_path + input_postcode[:2] + '.csv'
    # read in file
    postcode_df = pd.read_csv(postcode_file, names=['postcode', 'pqi', 'easting', 'northing'], header=None, usecols=[0, 1, 2, 3])
    #filter to input postcode
    postcode_df = postcode_df[postcode_df['postcode'] == input_postcode]
    # convert easting and northing to longitude and latitude
    postcode_df['longitude'], postcode_df['latitude'] = convert_easting_northing_to_long_lat(postcode_df['easting'].iloc[0], postcode_df['northing'].iloc[0])
    return postcode_df

# test function
post_df = get_postcode_data('NG1 1AA', postcode_path)
post_df

,postcode,pqi,easting,northing,longitude,latitude
0,NG1 1AA,10,457804,340087,52.955073,-1.141027


In [11]:
importlib.reload(sys.modules['src.geoapify_api'])
lat = post_df['latitude'].iloc[0]
long = post_df['longitude'].iloc[0]
request_params = {
    "apiKey": "493bbfea1f124b27906cf052aa330680",
    "categories": "tourism, leisure, entertainment",
    "limit": "500",
}
filters = {
        "circle": f"{lat},{long},5000",
}

url = 'https://api.geoapify.com/v2/places?'

df, res = geoapify_pipeline(url, request_params, filters)
df

,name,country,country_code,state,city,postcode,district,suburb,street,lon,...,state_code,formatted,address_line1,address_line2,county,town,license,url,website,category_keywords
0,National Justice Museum,United Kingdom,gb,England,Nottingham,NG1 1HN,Lace Market,The Meadows,High Pavement,-1.144218,...,ENG,"National Justice Museum, High Pavement, Nottin...",National Justice Museum,"High Pavement, Nottingham, NG1 1HN, United Kin...",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,https://www.nationaljusticemuseum.org.uk/museum/,"[building, wheelchair, tourism, entertainment,..."
1,Wollaton Hall,United Kingdom,gb,England,Nottingham,NG8 2AE,NaN,Wollaton Vale,Lime Tree Avenue,-1.209771,...,ENG,"Wollaton Hall, Lime Tree Avenue, Nottingham, N...",Wollaton Hall,"Lime Tree Avenue, Nottingham, NG8 2AE, United ...",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,https://www.wollatonhall.org.uk,"[building, tourism, entertainment, fee, museum]"
2,St Ann's Adventure Play Centre,United Kingdom,gb,England,Nottingham,NG3 3NB,NaN,St Ann's,Nugent Gardens,-1.136563,...,ENG,"St Ann's Adventure Play Centre, 95 Nugent Gard...",St Ann's Adventure Play Centre,"95 Nugent Gardens, Nottingham, NG3 3NB, United...",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,http://www.nottinghamcity.gov.uk/stannsadventure,"[entertainment, activity_park, leisure, playgr..."
3,Stanhope School Play Area,United Kingdom,gb,England,Gedling,NG4 4JB,NaN,Gedling,Shelford Road,-1.09776,...,ENG,"Stanhope School Play Area, Shelford Road, Gedl...",Stanhope School Play Area,"Shelford Road, Gedling, NG4 4JB, United Kingdom",Nottinghamshire,Carlton,Open Database Licence,https://www.openstreetmap.org/copyright,http://www.gedling.gov.uk/leisure/parksopenspa...,"[entertainment, activity_park, leisure, playgr..."
4,Gedling School Play Area,United Kingdom,gb,England,Gedling,NG4 4HX,NaN,Gedling,Wollaton Avenue,-1.092036,...,ENG,"Gedling School Play Area, Wollaton Avenue, Ged...",Gedling School Play Area,"Wollaton Avenue, Gedling, NG4 4HX, United Kingdom",Nottinghamshire,Carlton,Open Database Licence,https://www.openstreetmap.org/copyright,http://www.gedling.gov.uk/leisure/parksopenspa...,"[entertainment, activity_park, leisure, playgr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,Hyson Green,United Kingdom,gb,England,Nottingham,NaN,NaN,Hyson Green,NaN,-1.166938,...,ENG,"Hyson Green, Nottingham, England, United Kingdom",Hyson Green,"Nottingham, England, United Kingdom",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,NaN,"[park, leisure]"
460,Holme Pierrepont,United Kingdom,gb,England,Rushcliffe,NaN,NaN,NaN,NaN,-1.087102,...,ENG,"Holme Pierrepont, Rushcliffe, Nottinghamshire,...",Holme Pierrepont,"Rushcliffe, Nottinghamshire, England, United K...",Nottinghamshire,West Bridgford,Open Database Licence,https://www.openstreetmap.org/copyright,NaN,"[park, leisure]"
464,Colwick,United Kingdom,gb,England,Gedling,NaN,NaN,NaN,NaN,-1.088307,...,ENG,"Colwick, Gedling, Nottinghamshire, England, Un...",Colwick,"Gedling, Nottinghamshire, England, United Kingdom",Nottinghamshire,Carlton,Open Database Licence,https://www.openstreetmap.org/copyright,NaN,"[park, leisure]"
475,Manderley,United Kingdom,gb,England,Gedling,NaN,NaN,Woodthorpe,NaN,-1.109717,...,ENG,"Manderley, Woodthorpe, Gedling, Nottinghamshir...",Manderley,"Woodthorpe, Gedling, Nottinghamshire, England,...",Nottinghamshire,Arnold,Open Database Licence,https://www.openstreetmap.org/copyright,NaN,"[park, leisure]"


In [12]:
# In category column filter to look for ['tourism,'theatre'] and remove ['artwork','memorial']
df1 = df[df['category_keywords'].apply(lambda x: 'tourism' in x or 'theatre' in x)]
df1 = df1[df1['category_keywords'].apply(lambda x: 'artwork' not in x and 'memorial' not in x)]
df1

,name,country,country_code,state,city,postcode,district,suburb,street,lon,...,state_code,formatted,address_line1,address_line2,county,town,license,url,website,category_keywords
0,National Justice Museum,United Kingdom,gb,England,Nottingham,NG1 1HN,Lace Market,The Meadows,High Pavement,-1.144218,...,ENG,"National Justice Museum, High Pavement, Nottin...",National Justice Museum,"High Pavement, Nottingham, NG1 1HN, United Kin...",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,https://www.nationaljusticemuseum.org.uk/museum/,"[building, wheelchair, tourism, entertainment,..."
1,Wollaton Hall,United Kingdom,gb,England,Nottingham,NG8 2AE,NaN,Wollaton Vale,Lime Tree Avenue,-1.209771,...,ENG,"Wollaton Hall, Lime Tree Avenue, Nottingham, N...",Wollaton Hall,"Lime Tree Avenue, Nottingham, NG8 2AE, United ...",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,https://www.wollatonhall.org.uk,"[building, tourism, entertainment, fee, museum]"
8,Nottingham Castle Museum & Art Gallery,United Kingdom,gb,England,Nottingham,NG1 6EL,Lace Market,The Meadows,Castle Road,-1.154333,...,ENG,"Nottingham Castle Museum and Art Gallery, Cast...",Nottingham Castle Museum and Art Gallery,"Castle Road, Nottingham, NG1 6EL, United Kingdom",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,https://www.nottinghamcastle.org.uk/,"[building, tourism, entertainment, historic, s..."
11,Stonebridge City Farm,United Kingdom,gb,England,Nottingham,NG3 2FR,NaN,St Ann's,Ferrers Walk,-1.133927,...,ENG,"Stonebridge City Farm, Ferrers Walk, Nottingha...",Stonebridge City Farm,"Ferrers Walk, Nottingham, NG3 2FR, United Kingdom",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,https://stonebridgecityfarm.com/,"[attraction, tourism]"
12,Ye Olde Trip to Jerusalem,United Kingdom,gb,England,Nottingham,NG1 6AD,Lace Market,The Meadows,Castle Road,-1.152649,...,ENG,"Ye Olde Trip to Jerusalem, Castle Road, Nottin...",Ye Olde Trip to Jerusalem,"Castle Road, Nottingham, NG1 6AD, United Kingdom",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,http://www.triptojerusalem.com/,"[building, historic, tourism, heritage, cateri..."
17,Nottingham Playhouse,United Kingdom,gb,England,Nottingham,NG1 5AL,NaN,The Park,Wellington Circus,-1.156158,...,ENG,"Nottingham Playhouse, Wellington Circus, Notti...",Nottingham Playhouse,"Wellington Circus, Nottingham, NG1 5AL, United...",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,https://www.nottinghamplayhouse.co.uk/,"[entertainment, culture, building, theatre]"
19,Albert Hall,United Kingdom,gb,England,Nottingham,NG1 5AA,NaN,The Park,North Circus Street,-1.156231,...,ENG,"Albert Hall, North Circus Street, Nottingham, ...",Albert Hall,"North Circus Street, Nottingham, NG1 5AA, Unit...",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,http://www.alberthallnottingham.co.uk/,"[culture, building, historic, entertainment, h..."
20,Nottingham Contemporary,United Kingdom,gb,England,Nottingham,NG1 2GB,Lace Market,The Meadows,Weekday Cross,-1.145929,...,ENG,"Nottingham Contemporary, Weekday Cross, Nottin...",Nottingham Contemporary,"Weekday Cross, Nottingham, NG1 2GB, United Kin...",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,https://www.nottinghamcontemporary.org/,"[culture, building, wheelchair, tourism, enter..."
21,Pitcher & Piano,United Kingdom,gb,England,Nottingham,NG1 1HN,Lace Market,The Meadows,High Pavement,-1.145351,...,ENG,"Pitcher & Piano, High Pavement, Nottingham, NG...",Pitcher & Piano,"High Pavement, Nottingham, NG1 1HN, United Kin...",NaN,NaN,Open Database Licence,https://www.openstreetmap.org/copyright,http://www.pitcherandpiano.com/where-are-we/no...,"[building, church, wheelchair, historic, touri..."
22,The Nottingham New Theatre,United Kingdom,gb,England,Nottingham,NG7 2QH,NaN,Dunkirk,Cherry Tree Hill,-1.196123,...,ENG,"The Nottingham New Theatre, Cherry Tree Hill, ...",Th

In [6]:
# Check NaN rate for each column >0
nan_cols = df.isna().sum()[df.isna().sum()>50]
nan_cols

district    101
county      158
town        157
dtype: int64